<a href="https://colab.research.google.com/github/aminbaybon/Amazon-Imdb-review-classification-using-linearSVC/blob/main/Amazon_Imdb_review_classification_using_linearSVC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
import numpy as np
import pandas as pd
import string
from nltk.corpus import stopwords
import nltk
from spacy.lang.en.stop_words import STOP_WORDS
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.stem.porter import PorterStemmer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

In [17]:
imdb_Datast = pd.read_csv('imdb-labelled.txt' , sep='\t' , quoting=3 ,header=None)
'''
In pandas' read_csv() function, the parameter header is used to indicate which row should be considered
as the header (column names) of the DataFrame. By default, header is set to 'infer', which means pandas
will try to automatically detect the header row.
However, there are cases where you might want to explicitly specify that there is no header row in the
CSV file, and that the data should be loaded without considering the first row as the column names.
In such cases, you can set header=None
'''

the quoting parameter is used in the pd.read_csv() function from the pandas library to specify how to handle quoting characters in the CSV (Comma-Separated Values) file being read.


---


**quoting = 0**: Quote minimal. In this mode, no special handling for quoting characters is performed. All quoting characters (e.g., double quotes) are treated as regular characters, and they will be included in the fields as part of the data.


---



**quoting = 1**: Quote all. This mode treats all the data between the quoting characters as a single field, even if it contains delimiters (e.g., commas) within it. For example, if a field is enclosed in double quotes like "This is a field, with a comma", it will be considered as a single field.


---



**quoting = 2**: Quote non-numeric. This mode is similar to quoting = 1, but it only quotes fields that contain non-numeric data. Fields containing only numeric data are not quoted.


---



**quoting = 3**: Quote none. This mode disables quoting altogether. It is useful when there are no quoting characters present in the CSV file, and all fields are separated by delimiters (e.g., tabs, commas) directly.

In [15]:
amazon_data = pd.read_csv('amazon-cells-labelled.txt' , sep='\t' , quoting=3 ,header=None)

In [18]:
imdb_Datast

,0,1
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1
...,...,...
995,I just got bored watching Jessice Lange take h...,0
996,"Unfortunately, any virtue in this film's produ...",0
997,"In a word, it is embarrassing.",0
998,Exceptionally bad!,0


In [16]:
amazon_data

,0,1
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1
...,...,...
995,The screen does get smudged easily because it ...,0
996,What a piece of junk.. I lose more calls on th...,0
997,Item Does Not Match Picture.,0
998,The only thing that disappoint me is the infra...,0


In [20]:
#merge 2 datasets
data_total = amazon_data.append(imdb_Datast , ignore_index=True)

<ipython-input-20-8e9fa22fbe2f>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_total = amazon_data.append(imdb_Datast , ignore_index=True)


In [21]:
data_total

,0,1
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1
...,...,...
1995,I just got bored watching Jessice Lange take h...,0
1996,"Unfortunately, any virtue in this film's produ...",0
1997,"In a word, it is embarrassing.",0
1998,Exceptionally bad!,0


In [67]:
#valuse of lables
data_total[1].value_counts()

0    1000
1    1000
Name: 1, dtype: int64

In [23]:
data_total.isnull().sum()

0    0
1    0
dtype: int64

In [32]:
nltk.download('stopwords')
'''
used to download the "stopwords" dataset from the Natural Language Toolkit (NLTK) library.
Stopwords are common words that are often removed from texts during text processing tasks because
they typically do not add significant meaning to the content of the text. Examples of stopwords
include "the," "is," "in," "and," "that," etc.
'''

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


'\nused to download the "stopwords" dataset from the Natural Language Toolkit (NLTK) library.\nStopwords are common words that are often removed from texts during text processing tasks because\nthey typically do not add significant meaning to the content of the text. Examples of stopwords\ninclude "the," "is," "in," "and," "that," etc.\n'

In [24]:
x = data_total[0]
y = data_total[1]

In [26]:
punct = string.punctuation

punct = string.punctuation imports the string module and creates a variable punct containing all the punctuation characters defined in the string module.

The string module in Python provides a collection of commonly used string constants. One of these constants is punctuation, which is a string containing all the punctuation characters in the ASCII standard.

Punctuation characters typically include symbols such as commas, periods, exclamation marks, question marks, semicolons, colons, quotation marks, and other special characters used to punctuate sentences and phrases in written text.

In [27]:
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [38]:
stopwords2 = list(STOP_WORDS)

In [41]:
stopwords3 = list(stopwords.words('english'))

In [48]:
nlp = spacy.load('en_core_web_sm')
'''
nlp = spacy.load('en_core_web_sm') loads the English language pipeline from the spaCy library.
spaCy is an open-source library for advanced natural language processing in Python.
It provides pre-trained models and language pipelines to process text data efficiently.
The model 'en_core_web_sm' is a small English language model, which includes the vocabulary,
syntax, and word vectors necessary for basic NLP tasks.
'''
def clean1(sentence):
    text = nlp(sentence)
    '''
    text = nlp(sentence): This line processes the input sentence using the spaCy pipeline.
    It tokenizes the sentence, performs part-of-speech tagging, and other NLP tasks to
    create a Doc object containing the processed text.
    '''
    tokens = []
    '''
    Initialize an empty list to store the tokens extracted from the processed text.
    '''
    for t in text:
        if t.lemma_ != "-PRON-":
            '''
            Checks if the lemma (base form) of the token is not "-PRON-" (which stands for pronoun).
            '''
            temp = t.lemma_.lower().strip()
            '''
            If the token is not a pronoun, it converts the lemma to lowercase and removes any
            leading/trailing whitespace.The strip() method is used to remove leading and
            trailing whitespace characters (spaces, tabs, and newlines) from a string.
            It does not modify the original string but returns a new string with the l
            eading and trailing whitespace removed.
            '''
        else:
            temp = t.lower_
            '''
             If the token is a pronoun, it directly converts the token to lowercase.
             '''
        tokens.append(temp)

    clean_tokens = []
    for t in tokens:
        if t not in stopwords2 and t not in punct and t not in stopwords3:
            clean_tokens.append(t)
    return clean_tokens



In [49]:
clean1("helllllllo, it's god day!")

['helllllllo', 'god', 'day']

## **TF-IDF**

In [58]:
x_train, x_test, y_train , y_test = train_test_split(x,y,test_size=0.2 , random_state=0 , shuffle=True)

In [60]:
clf = Pipeline([('tfidf',TfidfVectorizer(tokenizer = clean1)),('clf' ,LinearSVC())])
'''
By using the pipeline, the text data will be automatically transformed into TF-IDF feature vectors
using the TfidfVectorizer and then used to train the LinearSVC classifier
The tokenizer parameter is set to clean1, which is a custom function used to preprocess the text
before vectorizing it. The clean1 function appears to perform text cleaning, lemmatization,
and tokenization, which helps in creating meaningful features for the classifier.
'''

In [61]:
clf

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function clean1 at 0x7da382864790>)),
                ('clf', LinearSVC())])

In [62]:
clf.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function clean1 at 0x7da382864790>)),
                ('clf', LinearSVC())])

In [63]:
y_pred = clf.predict(x_test)

In [66]:
accuracy_score(y_test,y_pred)

0.815

## **CountVectorizer**

In [68]:
clf2 = Pipeline([('tfidf',CountVectorizer(tokenizer = clean1)),('clf' ,LinearSVC())])

In [69]:
clf2

Pipeline(steps=[('tfidf',
                 CountVectorizer(tokenizer=<function clean1 at 0x7da382864790>)),
                ('clf', LinearSVC())])

In [70]:
clf2.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('tfidf',
                 CountVectorizer(tokenizer=<function clean1 at 0x7da382864790>)),
                ('clf', LinearSVC())])

In [71]:
y_pred2 = clf2.predict(x_test)

In [72]:
accuracy_score(y_test,y_pred2)

0.825

# **Notice**

In text analysis, the choice between CountVectorizer and TfidfVectorizer depends on the specific characteristics of the dataset and the machine learning task at hand. In some cases, CountVectorizer may lead to better results, while in other cases, TfidfVectorizer might be more suitable.
**CountVectorizer** is useful when the **absolute occurrence of words is more** **important than their relative importance across documents**. For example, in tasks where the frequency of words is a crucial feature (e.g., spam vs. non-spam classification), CountVectorizer might perform well.

**TfidfVectorizer** is useful when you want to **emphasize the importance of words that are frequent within a document but not frequent across all documents**. For tasks where the uniqueness of words in each document matters (e.g., sentiment analysis, topic modeling), TfidfVectorizer might perform better.

**When CountVectorizer might outperform TfidfVectorizer**:

The task focuses on individual document characteristics, and the relative frequency of words across documents is not critical.
Tasks where the occurrence of specific words is highly indicative of the target variable (e.g., detecting specific keywords).

**When TfidfVectorizer might outperform CountVectorizer**:

The task requires capturing the importance of words within documents while accounting for the frequency of words across documents.
Tasks where unique words and their distribution are more informative than the absolute occurrence of words.